<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Fallstudie: Othellite</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>

In [ ]:
from collections.abc import Sequence
from enum import Enum
from dataclasses import dataclass, field
import reprlib
from typing import Union
from random import sample

try:
    import numpy as np
except ModuleNotFoundError:
    print("NumPy not found, some (minor) examples may not work")


# Othellite

Im Folgenden wollen wir eine vereinfachte Variante des Spiels Reversi (auch
unter dem Handelsnamen Othello bekannt) implementieren. Das Spiel wird auf
einem Brett mit 8x8 Feldern gespielt, auf die Spieler schwarze oder
weiße Spielsteine legen können. Die genauen Regeln sind
[im Wikipedia Artikel](https://de.wikipedia.org/wiki/Othello_(Spiel))
beschrieben.


Wir definieren eine Enumeration, die den Zustand eines einzelnen Feldes auf
dem Brett beschreibt:

In [ ]:
print("Fields:           ", list(Field))
print("Empty field:      ", Field.DARK)
print("Light field:      ", Field.LIGHT)
print("Dark field:       ", Field.DARK)
print("Light field value:", Field.LIGHT.value)
print("Light field name: ", Field.LIGHT.name)


## Mini-Workshop

Definieren Sie eine Funktion `is_occupied(f: Field) -> bool`,
die genau dann `True` zurückgibt, wenn `field` nicht leer ist.

In [ ]:
def is_occupied(f: Field) -> bool:
    return f is not Field.EMPTY

In [ ]:
assert is_occupied(Field.DARK)
assert is_occupied(Field.LIGHT)
assert not is_occupied(Field.EMPTY)


## Mini-Workshop

Definieren Sie eine Enumeration `Player`, die analog zu `Field` die beiden
Spieler beschreibt. Drucken Sie ähnlich wie für `Field` die möglichen Werte
sowie Value und Namen für einen Enum-Wert aus.


## Mini-Workshop

Definieren Sie eine Funktion
`is_field_owned_by_opponent(p: Player, f: Field) -> bool`,
die genau dann `True` zurückgibt, wenn der Spielstein auf Feld `f` dem
Gegner gehört, d.h., wenn `f` belegt ist und der Stein auf `f` nicht die
gleiche Farbe hat wie der Spieler.

Überprüfen Sie, dass Ihre Implementierung die angegebenen Assertions erfüllt.

Implementieren Sie eine entsprechende Methode `is_field_owned_by_player()`.

In [ ]:
assert is_field_owned_by_opponent(Player.LIGHT, Field.DARK)
assert is_field_owned_by_opponent(Player.DARK, Field.LIGHT)
assert not is_field_owned_by_opponent(Player.DARK, Field.DARK)
assert not is_field_owned_by_opponent(Player.DARK, Field.EMPTY)
assert not is_field_owned_by_opponent(Player.LIGHT, Field.LIGHT)
assert not is_field_owned_by_opponent(Player.LIGHT, Field.EMPTY)

In [ ]:
assert is_field_owned_by_player(Player.LIGHT, Field.LIGHT)
assert is_field_owned_by_player(Player.DARK, Field.DARK)
assert not is_field_owned_by_player(Player.DARK, Field.LIGHT)
assert not is_field_owned_by_player(Player.DARK, Field.EMPTY)
assert not is_field_owned_by_player(Player.LIGHT, Field.DARK)
assert not is_field_owned_by_player(Player.LIGHT, Field.EMPTY)


Wir wollen auf die einzelnen Felder eines 8x8 Spielfelds mittels eines
zweidimensionalen Index-Werts zugreifen: `board[0, 0]` steht für das linke
obere Feld, `board[0, 1]` für das Feld rechts daneben, usw.

Das ist einfach, wenn wir ein zweidimensionales Array als zugrunde liegende
Datenstruktur verwenden können, z.B. das `ndarray` von NumPy. In diesem Fall
delegieren wir einfach `__getitem__()` und `__setitem__()` an die
entsprechenden Methoden des NumPy Arrays.

Die mittleren 4 Felder des Spielfelds initialisieren wir mit diagonal
angeordneten Steinen.

<!--
We want to be able to access the individual fields of the 8x8 board using
a 2-dimensional index notation: `board[0, 0]` for the field in the upper
left corner, `board[0, 1]` for the field to the right of the upper left
corner, etc.

This is easy when using a two-dimensional array, such as NumPy's `ndarray`.
We simply need to delegate the `__getitem__()` and `__setitem__()` methods
to the NumPy array:
-->


Dadurch, dass der Index an das NumPy Array weitergereicht wird, stehen auch
mächtigere Zugriffsvarianten, wie z.B. Slicing zur Verfügung:

In [ ]:
npb[0, 2:] = Field.DARK
npb[1:3, 1:3] = Field.LIGHT
print(npb)

## Mini-Workshop Compute Linear Index

Wenn wir eine Python Liste als Speicher für das Spielfeld verwenden wollen,
müssen wir die Umrechnung von zweidimensionalen Index-Werten in einen
eindimensionalen Index selber vornehmen.

Das geht indem wir für eine Position `(r, c)` den Wert `r * 8 + c` berechnen.
Für den Fall, dass einer der Indices nicht `>= 0` und `< 8` ist soll eine
geeignete Exception ausgelöst werden.

Implementieren Sie eine Funktion `compute_linear_index()` die diese Aufgabe
übernimmt.


## Mini-Workshop

Implementieren Sie eine Klasse `Board`, die ein 8x8 Othellite Brett
repräsentiert, das den Zustand des Bretts in einer Liste speichert.
Sie können sich bei der Implementierung an der Klasse `NumPyBoard`
orientieren, und `compute_linear_index` zur Berechnung von Index-Werten
verwenden.


Wir definieren eine weitere Enumeration für "Himmelsrichtungen". Die
Schlüssel der Enumeration sollen Abkürzungen für die Kompassrichtungen (N,
NE, E, ...) sein. Als Wert soll jeweils der zur Bewegung in diese Richtung
benötigte "Offset" verwendet werden:

- `(-1, 0)` für eine Bewegung in Richtung Norden,
- `(0, 1)` für eine Bewegung in Richtung Osten,
- `(1, 0)` für eine Bewegung in Richtung Süden,
- `(0, -1)` für eine Bewegung in Richtung Westen,
- `(-1, 1)` für eine Bewegung in Richtung Nord-Osten,
- `(1, 1)` für eine Bewegung in Richtung Süd-Osten,
- `(-1, -1)` für eine Bewegung in Richtung Nord-Westen,
- `(1, -1` für eine Bewegung in Richtung Süd-Westen.

<!--
We define yet another enumeration, this time for directions.
The keys of this enumeration should be the compass directions (N, NE, E, ...),
the values should be the (x, y) offset to move to this field in a
right-handed coordinate system, i.e.,
- `(-1, 0)` for moving north,
- `(0, 1)` for moving east,
- `(1, 0)` for moving south,
- `(0, -1)` for moving west,
- `(-1, 1)` for moving north-east,
- `(1, 1)` for moving south-east,
- `(-1, -1)` for moving north-west,
- `(1, -1` for moving south-west.
-->


## Mini-Workshop

Definieren Sie eine Enumeration `Directions`, wie gerade beschrieben.

Es ist bei Enumerationen möglich, mehrere Schlüssel für den gleichen Wert
zu definieren (also gewissermaßen Synonyme anzugeben).

```python
class Synonyms(Enum):
    DRINK = 0
    BEVERAGE = 0
    FOOD = 1
    BIG = 2
    LARGE = 2
    SMALL = 3
```

In [ ]:
print(Synonyms.DRINK is Synonyms.BEVERAGE)
print(Synonyms.DRINK is not Synonyms.FOOD)
print(Synonyms.BIG is Synonyms.LARGE)
print(Synonyms.BIG is not Synonyms.SMALL)


Erweitern Sie die Enumeration `Directions` so, dass auch die vollständigen
Namen der Kompassrichtungen (`NORTH`, `NORTH_EAST`, ...) verwendet werden
können.


## Mini-Workshop

Überprüfen Sie, dass die Abkürzungen und die vollständigen Name die
gleichen Werte repräsentieren.
<!--
Ensure that the abbreviated names and the full names denote the
same values.
-->

In [ ]:
Index = tuple[int, int]

## Mini-Workshop

Schreiben Sie eine Funktion `is_valid_index(index: Index) -> bool`,
die genau dann `True` zurückgibt, wenn `index` ein gültiger Index für
ein Othellite Brett ist, andernfalls `False`. Überprüfen Sie, ob Ihre
Implementierung die Assertions erfüllt

In [ ]:
assert is_valid_index((0, 7))
assert not is_valid_index((8, 1))
assert not is_valid_index((1, 8))
assert not is_valid_index((6, -1))
assert not is_valid_index((-2, 4))

## Mini-Workshop

Schreiben Sie eine Funktion
`assert_valid_index(index: Index) -> None:`, die eine Exception vom
Typ `IndexError` mit einer geeigneten Fehlermeldung auslöst, wenn `index`
kein gültiger Index für ein Othellite-Brett ist. Überprüfen Sie, dass die
Funktion für korrekte Argumente keinen Exception auslöst, dass sie aber für
verschiedene Fehlerfälle eine Exception auslöst.


Schreiben Sie eine Funktion
`next_index_in_direction(index: Index, direction: Direction)`,
die zwei Werte zurückgibt, falls `index` ein gültiger Index ist:
- den nächsten Index in Richtung `direction` und
- `True` oder `False`, je nachdem ob der nächste Index gültig ist oder nicht.
Falls `index` kein gültiger Index ist soll eine Exception vom Typ
`IndexError` ausgelöst werden.

*Hinweis:* Die Werte, die zu den Komponenten von `index` addiert werden
müssen können Sie mittels `d_row, d_column = direction.value` berechnen.

Überprüfen Sie, ob Ihre Implementierung die angegebenen Assertions erfüllt.

In [ ]:
assert next_index_in_direction((0, 0), Direction.N) == ((-1, 0), False)
assert next_index_in_direction((0, 0), Direction.NE) == ((-1, 1), False)
assert next_index_in_direction((0, 0), Direction.E) == ((0, 1), True)
assert next_index_in_direction((0, 0), Direction.SE) == ((1, 1), True)
assert next_index_in_direction((0, 0), Direction.S) == ((1, 0), True)
assert next_index_in_direction((0, 0), Direction.SW) == ((1, -1), False)
assert next_index_in_direction((0, 0), Direction.W) == ((0, -1), False)
assert next_index_in_direction((0, 0), Direction.NW) == ((-1, -1), False)

In [ ]:
assert next_index_in_direction((4, 5), Direction.N) == ((3, 5), True)
assert next_index_in_direction((4, 5), Direction.NE) == ((3, 6), True)
assert next_index_in_direction((4, 5), Direction.E) == ((4, 6), True)
assert next_index_in_direction((4, 5), Direction.SE) == ((5, 6), True)
assert next_index_in_direction((4, 5), Direction.S) == ((5, 5), True)
assert next_index_in_direction((4, 5), Direction.SW) == ((5, 4), True)
assert next_index_in_direction((4, 5), Direction.W) == ((4, 4), True)
assert next_index_in_direction((4, 5), Direction.NW) == ((3, 4), True)

In [ ]:
assert next_index_in_direction((7, 7), Direction.N) == ((6, 7), True)
assert next_index_in_direction((7, 7), Direction.NE) == ((6, 8), False)
assert next_index_in_direction((7, 7), Direction.E) == ((7, 8), False)
assert next_index_in_direction((7, 7), Direction.SE) == ((8, 8), False)
assert next_index_in_direction((7, 7), Direction.S) == ((8, 7), False)
assert next_index_in_direction((7, 7), Direction.SW) == ((8, 6), False)
assert next_index_in_direction((7, 7), Direction.W) == ((7, 6), True)
assert next_index_in_direction((7, 7), Direction.NW) == ((6, 6), True)

In [ ]:
try:
    next_index_in_direction((7, 8), Direction.S)
except IndexError as err:
    print(err)


Wir wollen unsere Klasse `Board` jetzt um eine Methode

```
_fields_flipped_in_direction(self, p: Player, index: Index,
                             d: Direction) -> set[Index]
```

erweitern, die die Indizes aller Felder zurückgibt, die der Spieler ausgehend
vom Feld `index` in Richtung `d` umdrehen kann.

In [ ]:
assert _find_rightmost([True, False, True, False]) == 2
assert _find_rightmost([False, False, False]) == 0

In [ ]:
def assert_flips_direction(player, index, directions):
    board = Board()
    board[index] = Field.LIGHT if player == Player.LIGHT else Field.DARK
    print(f"Player {player} at {index} flips {directions}.")
    print(board)

    for d in set(Direction) - directions:
        # noinspection PyProtectedMember
        result = board._indices_to_flip_in_direction(player, index, d)
        assert result == set(), f"Fields flipped in {d} are not empty."

    for d in directions:
        # noinspection PyProtectedMember
        result = board._indices_to_flip_in_direction(player, index, d)
        assert result != set(), f"Fields flipped in {d} are empty."

In [ ]:
assert_flips_direction(Player.LIGHT, (0, 0), set())
assert_flips_direction(Player.LIGHT, (2, 3), {Direction.S})
assert_flips_direction(Player.LIGHT, (3, 2), {Direction.E})
assert_flips_direction(Player.LIGHT, (4, 5), {Direction.W})
assert_flips_direction(Player.LIGHT, (5, 2), set())
assert_flips_direction(Player.LIGHT, (5, 3), set())
assert_flips_direction(Player.LIGHT, (5, 4), {Direction.N})
assert_flips_direction(Player.LIGHT, (6, 4), set())

In [ ]:
assert_flips_direction(Player.DARK, (0, 0), set())
assert_flips_direction(Player.DARK, (2, 4), {Direction.S})
assert_flips_direction(Player.DARK, (3, 5), {Direction.W})
assert_flips_direction(Player.DARK, (4, 2), {Direction.E})
assert_flips_direction(Player.DARK, (5, 2), set())
assert_flips_direction(Player.DARK, (5, 3), {Direction.N})
assert_flips_direction(Player.DARK, (5, 4), set())
assert_flips_direction(Player.DARK, (6, 4), set())


## Mini-Workshop

Erweitern Sie die Klasse `Board` um eine Methode
`is_valid_move(self, player: Player, index: Index) -> bool`
die genau dann `True` zurückgibt, wenn `index` eine gültiger Index für
den nächsten Zug des Spielers `player` ist.

Ein Index ist gültig, wenn
- Er nicht schon besetzt ist,
- Mindestens ein Stein des Gegenspielers durch den Zug umgedreht wird


In [ ]:
def assert_valid_moves(player, valid_indices):
    board = Board()
    for row in range(8):
        for col in range(8):
            index = (row, col)
            result = board.is_valid_move(player, index)
            if index in valid_indices:
                assert result, f"Index {index} should be valid."
            else:
                assert not result, f"Index {index} is valid?"

In [ ]:
print(Board())

In [ ]:
valid_moves_for_light_player = {(2, 3), (3, 2), (4, 5), (5, 4)}
assert_valid_moves(Player.LIGHT, valid_moves_for_light_player)

In [ ]:
valid_move_for_dark_player = {(2, 4), (3, 5), (4, 2), (5, 3)}
assert_valid_moves(Player.DARK, valid_move_for_dark_player)


## Mini-Workshop

Erweitern Sie die Klasse `Board` um eine Methode
`find_valid_moves(self, player: Player) -> set[Index]`,
die alle für den Spieler `player` möglichen Züge zurückgibt.

*Hinweis:* Das können Sie leicht mittels *Generate and Test* erreichen:
Erzeugen Sie alle möglichen Züge und testen Sie dann für jeden Zug, ob
er möglich ist.

In [ ]:
board = Board()
assert board.find_valid_moves(Player.LIGHT) == valid_moves_for_light_player
assert board.find_valid_moves(Player.DARK) == valid_move_for_dark_player


## Mini-Workshop

Schreiben Sie eine Methode
`play_move(self, player: Player, index: Index) -> None`,
die den Zug von `player` auf das Feld `index` ausführt.
Falls der Zug ungültig ist, soll eine Exception ausgelöst werden.
Falls der Zug gültig ist, sollen alle Felder, die durch den Zug
umgedreht werden von dieser Methode auf den korrekten Wert gesetzt werden.

Schreiben Sie dazu geeignete Hilfsmethoden, damit `play_move` nicht zu
viele Verantwortlichkeiten hat.

In [ ]:
def play_some_random_moves(n: int = 10):
    board = Board()
    player = Player.DARK
    for _ in range(n):
        print(board)
        moves = board.find_valid_moves(player)
        if moves:
            move = sample(list(moves), 1)[0]
            print(f"Player {player} plays {move}.")
            board.play_move(player, move)
        else:
            print(f"{player} has no more valid moves.")
        player = Player.LIGHT if player == Player.DARK else Player.DARK
    print(board)

In [ ]:
play_some_random_moves()